In [1]:
%load_ext autoreload
%autoreload 2

from util_0804_6 import *
from myalgorithm_0804_6 import algorithm

import pandas as pd

## 실험 내용
1. 4주문 번들 SOTA 8-3_bundling_opt_ver1_KruskalRandom_KCasework로 WALK 확인

### 모든 데이터셋 실험

In [2]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

cts = []
for problem_file in problem_files:
    start_time = time.time()

    testcase_name = problem_file.split('\\')[-1].split('.')[0]

    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## --------------------- 커스텀 코드 시작 ---------------------------

    solution, result_bundles = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

    end_time = time.time()
    process_time = end_time - start_time

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    # ---------------------- WALK 확인용 코드 ----------------------------

    used_rider_info = [(v.rider.type, len(v.shop_seq)) for v in result_bundles]

    ct = [[0, 0] for _ in range(6)] # ct[order_c] = [walk_c, non_walk_c]
    for rider_type, order_c in used_rider_info:
        if rider_type == 'WALK':
            ct[order_c][0] += 1
        else:
            ct[order_c][1] += 1

    cts.append(ct)

    print(testcase_name, process_time, checked_solution['avg_cost'], ct)

TEST_K50_1 55.78734087944031 3557.0260000000003 [[0, 0], [5, 1], [0, 4], [0, 8], [0, 3], [0, 0]]
TEST_K50_2 55.787193059921265 3988.3459999999995 [[0, 0], [4, 2], [0, 8], [0, 8], [0, 1], [0, 0]]
TEST_K100_1 55.85739874839783 3460.855 [[0, 0], [0, 3], [1, 11], [0, 19], [0, 4], [0, 0]]
TEST_K100_2 55.81309461593628 3582.788 [[0, 0], [5, 5], [0, 10], [0, 18], [0, 4], [0, 0]]
TEST_K200_1 56.210516929626465 3245.765 [[0, 0], [5, 3], [1, 19], [0, 32], [0, 14], [0, 0]]
TEST_K200_2 56.346033334732056 3223.146 [[0, 0], [4, 2], [0, 23], [0, 32], [0, 13], [0, 0]]
STAGE1_1 55.9782075881958 3699.8979999999992 [[0, 0], [0, 2], [0, 8], [0, 18], [0, 7], [0, 0]]
STAGE1_2 55.97719693183899 3291.82 [[0, 0], [0, 0], [0, 3], [0, 10], [0, 16], [0, 0]]
STAGE1_3 56.16830539703369 3043.3965 [[0, 0], [3, 0], [4, 15], [0, 29], [0, 18], [0, 0]]
STAGE1_4 56.29206442832947 2679.9609999999993 [[0, 0], [0, 0], [0, 4], [0, 12], [0, 39], [0, 0]]
STAGE1_5 56.56155467033386 2306.393000000001 [[0, 0], [3, 0], [6, 8], [0, 

In [ ]:
STAGE1_6 56.390345096588135 2483.517999999999 [[0, 0], [2, 0], [5, 7], [1, 9], [2, 59], [0, 0]]